# 📈 Stock market chart pattern #

In [29]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import plotly.graph_objects as go

In [30]:
# สุ่มแบบกำหนด seed เพื่อความสามารถในการทำซ้ำ
np.random.seed(42)

# ===== Step 1: สร้างข้อมูลย้อนหลัง 5 เดือน (ประมาณ 100 วันทำการ) =====
n_days = 100
start_price = 72.0
date_range = pd.bdate_range(start='2024-01-01', periods=n_days)  # วันทำการเท่านั้น

# สร้างราคาปิด (Close) แบบ random walk
daily_returns = np.random.normal(loc=0.0005, scale=0.01, size=n_days)
close_prices = start_price * np.cumprod(1 + daily_returns)

# สร้างราคาตลาดอื่น ๆ ให้ดูสมจริง
open_prices = [close_prices[0]]
for i in range(1, n_days):
    open_prices.append(close_prices[i - 1] * (1 + np.random.normal(0, 0.002)))

high_prices = [max(o, c) * (1 + np.random.uniform(0.001, 0.015)) for o, c in zip(open_prices, close_prices)]
low_prices = [min(o, c) * (1 - np.random.uniform(0.001, 0.015)) for o, c in zip(open_prices, close_prices)]

# Volume สมจริง
volumes = np.random.randint(1_000_000, 10_000_000, size=n_days)

# สร้าง DataFrame
df = pd.DataFrame({
    'Date': date_range,
    'Open': open_prices,
    'High': high_prices,
    'Low': low_prices,
    'Close': close_prices,
    'Volume': volumes,
    'Ticker': ['AOT'] * n_days
})

# ===== Step 2: ตรวจจับ Head and Shoulders บน Close =====
def detect_head_and_shoulders(df, ticker):
    df = df[df['Ticker'] == ticker].copy()
    df.reset_index(drop=True, inplace=True)

    peaks, _ = find_peaks(df['Close'], distance=5)
    valleys, _ = find_peaks(-df['Close'], distance=5)

    pattern_points = []

    for i in range(len(peaks) - 2):
        left = peaks[i]
        head = peaks[i+1]
        right = peaks[i+2]

        if left < head < right:
            if df['Close'][head] > df['Close'][left] and df['Close'][head] > df['Close'][right]:
                if abs(df['Close'][left] - df['Close'][right]) < df['Close'][head]*0.05:
                    pattern_points.append((left, head, right))

    return df, pattern_points




In [36]:
# ===== Step 3: วาด Candlestick Chart พร้อม Head and Shoulders =====
def plot_chart_with_pattern(df, pattern_points, ticker):
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=df['Date'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlestick'
    ))

    # วาดจุด Head and Shoulders
    for (l, h, r) in pattern_points:
        fig.add_trace(go.Scatter(
            x=[df['Date'][l]], y=[df['Close'][l]],
            mode='markers+text', name='Left Shoulder',
            marker=dict(color='yellow', size=10),
            text=["Left"], textposition="top center"
        ))
        fig.add_trace(go.Scatter(
            x=[df['Date'][h]], y=[df['Close'][h]],
            mode='markers+text', name='Head',
            marker=dict(color='red', size=12),
            text=["Head"], textposition="top center"
        ))
        fig.add_trace(go.Scatter(
            x=[df['Date'][r]], y=[df['Close'][r]],
            mode='markers+text', name='Right Shoulder',
            marker=dict(color='green', size=10),
            text=["Right"], textposition="top center"
        ))

        # neckline
        fig.add_shape(
            type='line',
            x0=df['Date'][l], y0=df['Close'][l],
            x1=df['Date'][r], y1=df['Close'][r],
            line=dict(color='green', dash='dash')
        )

    fig.update_layout(
        title=f'{ticker} - Head and Shoulders Pattern',
        xaxis_title='Date',
        yaxis_title='Price (Baht)',
        template='plotly_white',
        xaxis_rangeslider_visible=False,
        height=600
    )

    fig.show()


In [37]:
# Run detection + plot
df, patterns = detect_head_and_shoulders(df, 'AOT')
plot_chart_with_pattern(df, patterns, 'AOT')